In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("UngLong/cafebert-classification-ft")
model = AutoModelForSequenceClassification.from_pretrained("UngLong/cafebert-classification-ft")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset, Dataset
import numpy as np
import pandas as pd
import re
import unicodedata
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Load tập test
dataset = load_dataset("UngLong/ViSIR", split="test")

In [ ]:
# 1️⃣ Dictionary chứa từ viết tắt toxic
toxic_abbreviations = {
    "vl": "vãi lồn",
    "vkl": "vãi cả lồn",
    "dm": "đụ mẹ",
    "dmm": "đụ mẹ mày",
    "cl": "cái lồn",
    "vc": "vãi cặc",
    "đmm": "đụ mẹ mày",
    "cc": "con cặc",
    "lol": "lồn",
    "dkm": "địt con mẹ",
    "đcmm": "địt con mẹ mày",
    "wtf": "cái đéo gì vậy",
    "clmm": "cái lồn mẹ mày",
    "ccmm": "con chó mẹ mày",
    "dcm": "địt con mẹ mày",
    "m": "mày",
    "t": "tao",
    "vcl": "vãi cả lồn",
    "nguu": "ngu",
    "tk": "thằng",
    "djt": "địt",
    "lmm": "lồn mẹ mày",
    "lòn": "lồn",
    "óc c": "óc cặc",
    "clm": "cái lồn má",
    "djtme": "địt mẹ",
    "đcho": "đĩ chó",
    "iồn": "lồn"
}

# 2️⃣ Regex thay thế từ viết tắt toxic (không nhầm trong từ khác)
toxic_pattern = re.compile(r"\b(" + "|".join(map(re.escape, toxic_abbreviations.keys())) + r")(?=\W|$)", re.IGNORECASE)

# 3️⃣ Hàm mở rộng từ viết tắt
def expand_toxic_abbreviations(text):
    return toxic_pattern.sub(lambda m: toxic_abbreviations[m.group(1).lower()], text)

# 4️⃣ Hàm làm sạch văn bản
def clean_text(text):
    # Chuyển sang string nếu không phải string
    if not isinstance(text, str):
        text = str(text)

    # Chuẩn hóa Unicode
    text = unicodedata.normalize("NFC", text)

    # Loại bỏ emoji
    text = re.sub(r"[^\w\s\u00C0-\u1FFF\u2C00-\uD7FF]", " ", text)

    # Thay thế từ viết tắt toxic
    text = expand_toxic_abbreviations(text)

    # Xóa khoảng trắng thừa
    text = re.sub(r"\s+", " ", text)

    return text.lower().strip()


In [ ]:
df_test = pd.DataFrame(dataset)

In [ ]:
df_test['text'] = df_test['text'].apply(clean_text)

In [ ]:
dataset_test = Dataset.from_pandas(df_test)

In [ ]:
# Tokenize dữ liệu
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",  # Đảm bảo mọi mẫu đều có độ dài 77
        max_length=77
    )


In [ ]:
encoded_dataset = dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/10750 [00:00<?, ? examples/s]

In [ ]:
# Chỉ giữ lại các trường cần thiết và chuyển thành tensor
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Tạo DataLoader
test_dataloader = DataLoader(encoded_dataset, batch_size=16)

# Đánh giá mô hình
model.eval()
predictions = []
labels = []

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Di chuyển model lên GPU nếu có

with torch.no_grad():
    for batch in test_dataloader:
        # Kiểm tra kiểu dữ liệu của batch
        print({k: type(v) for k, v in batch.items()})  # Debug

        # Di chuyển dữ liệu lên thiết bị phù hợp (GPU nếu có)
        inputs = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
        label_batch = batch["label"].to(device)  # Đưa nhãn về thiết bị

        # Dự đoán
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        # Lưu lại dự đoán và nhãn
        predictions.extend(preds.cpu().numpy())  # Chuyển về CPU để tránh lỗi
        labels.extend(label_batch.cpu().numpy())

{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 

In [ ]:
# Tính toán độ chính xác
accuracy = accuracy_score(labels, predictions)
report = classification_report(labels, predictions, target_names=["Non-Toxic", "Toxic"])

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

Accuracy: 0.9227
Classification Report:
               precision    recall  f1-score   support

   Non-Toxic       0.95      0.90      0.92      5548
       Toxic       0.90      0.95      0.92      5202

    accuracy                           0.92     10750
   macro avg       0.92      0.92      0.92     10750
weighted avg       0.92      0.92      0.92     10750

